In [ ]:
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

### Pricing curve is assumed accurate

This curve only has the four instruments 2Y, 5Y, 10Y, 30Y, as per the text.

In [ ]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2024, 1, 1): Dual(1, {"v1": 1}),
    datetime(2027, 1, 1): Dual(1, {"v2": 1}),
    datetime(2032, 1, 1): Dual(1, {"v3": 1}),
    datetime(2052, 1, 1): Dual(1, {"v4": 1})
}
swaps = {
    Swap(datetime(2022, 1, 1), 12*2, 12, 12): 1.635,
    Swap(datetime(2022, 1, 1), 12*5, 12, 12): 1.885,
    Swap(datetime(2022, 1, 1), 12*10, 12, 12): 1.930,
    Swap(datetime(2022, 1, 1), 12*30, 12, 12): 1.980,
}
s_cv = SolvedCurve(
    nodes=nodes,
    swaps=list(swaps.keys()),
    obj_rates=list(swaps.values()),
    interpolation="log_linear",
    algorithm="levenberg_marquardt"
)
print(s_cv.iterate())

### Equivalent portfolios

Suppose we construct three portfolios, each with either;

  - A single 5Y5Y swap,
  - A 5Y swap and a 10Y swap,
  - A 5s10s swap spread and a 10Y swap,

These portfolios are **equivalent** since their cashflows are identical.


In [ ]:
portfolio1 = Portfolio([
    Swap(datetime(2027,1,1), 12*5, 12, 12, fixed_rate=1.9797, notional=-100e6)
])
portfolio2 = Portfolio([
    Swap(datetime(2022,1,1), 12*5, 12, 12, fixed_rate=1.9797, notional=100e6),
    Swap(datetime(2022,1,1), 12*10, 12, 12, fixed_rate=1.9797, notional=-100e6)
])
portfolio3 = Portfolio([
    Swap(datetime(2022,1,1), 12*5, 12, 12, fixed_rate=1.9797, notional=100e6),
    Swap(datetime(2022,1,1), 12*10, 12, 12, fixed_rate=1.9797, notional=-50e6),
    Swap(datetime(2022,1,1), 12*10, 12, 12, fixed_rate=1.9797, notional=-50e6)
])

df = pd.DataFrame({
    "pf1": portfolio1.risk(s_cv)[:,0],
    "pf2": portfolio2.risk(s_cv)[:,0],
    "pf3": portfolio3.risk(s_cv)[:,0],
}, index=["2y", "5y", "10y", "30y"])
df.style.format("{:,.0f}")

### Parametrised instruments

Next we use the prices and the insruments as parametrised in the text.

In [ ]:
df = pd.DataFrame({
    "2Y": [1, 0, 0, 0, -1, -1, -1, 0, 0, 0, -1, 0],
    "5Y": [0, 1, 0, 0, 1, 0, 0, -1, -1, 0, 2, -1],
    "10Y": [0, 0, 1, 0, 0, 1, 0, 1, 0, -1, -1, 2],
    "30Y": [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, -1],
    "margin": [0.4, 0.5, 0.5, 0.6, 0.25, 0.45, 0.85, 0.25, 0.7, 0.55, 0.5, 0.6],
}, index=["2Y", "5Y", "10Y", "30Y", "2s5s", "2s10s", "2s30s", "5s10s", "5s30s", "10s30s", "2s5s10s", "5s10s30s"])
df.style

In [ ]:
c = df["margin"].to_numpy()[:, np.newaxis]
A = df[["2Y", "5Y", "10Y", "30Y"]].to_numpy().T
S = portfolio1.risk(s_cv)
ret = portfolio1.model_margin(c, A, portfolio1.risk(s_cv))

In [ ]:
df["x"], x = ret[1], ret[1][:, np.newaxis]
df.style.format("{:,.1f}")

The solution to this linear program is the above 'x' column and the calculated margin for these instruments is, below, which equates to a margin, from mid, of 0.78bps on the delta of the underlying 5Y5Y IRS.

In [ ]:
ret[0]

### Volume

To account for the fact that our model is volume agnostic we will assume an additional hedge cost of:

$$ \frac{2}{3000} * \sqrt{x_i} * x_i $$

In [ ]:
df["basic margin"] = df["margin"] * np.abs(df["x"])
df["volume add on"] = 2/3000 * np.abs(df["x"])**1.5
df.style.format("{:,.1f}")

### Correlation of identified market hedges

The hedges that our model has identified will be assessed to see if they are correlated and if we need to charge extra.

In [ ]:
historical_rates = pd.DataFrame({
    "2Y": [1.199, 1.228, 1.210, 1.215, 1.203, 1.159, 1.175, 1.188, 1.159, 1.100],
    "5Y": [1.663, 1.696, 1.665, 1.680, 1.677, 1.657, 1.673, 1.676, 1.653, 1.600],
    "10Y": [1.928, 1.945, 1.934, 1.93, 1.934, 1.931, 1.958, 1.972, 1.932, 1.900],
    "30Y": [2.201, 2.217, 2.228, 2.239, 2.226, 2.235, 2.242, 2.236, 2.22, 2.200],
})
historical_chgs = historical_rates.diff(-1)*100
historical_chgs.style.format("{:,.1f}")
Q = historical_chgs.cov().to_numpy()
Q

In [ ]:
Q_model = np.matmul(np.matmul(A.T, Q), A)
df2 = pd.DataFrame(Q_model, index=df.index, columns=df.index)
df2.style.format("{:,.2f}")

#### CoVaR Multiplier
The covar multiplier for the trade is:

In [ ]:
covar = np.matmul(np.matmul(x.T, Q_model), x)[0][0] ** 0.5
covar

In [ ]:
covar_zerocorr = np.matmul(np.matmul(x.T, np.diag(np.diagonal(Q_model))), x)[0][0] ** 0.5
covar_zerocorr

The covar multiplier for the trade assuming completely uncorrelated instruments is:

The difference is quite small. We can observe from the covariance matrix that 10Y and 5s10s has low correlation.

### Liquidity Assessment

Assume the bund future represents our market, we calculate the intrinsic bid-ask width from different order books representing different conditions.

In [ ]:
b = np.array([169.70, 169.69, 169.68, 169.67, 169.66, 169.65])
a = np.array([169.71, 169.72, 169.73, 169.74, 169.75, 169.76])
w1 = np.array([36, 78, 86, 111, 121, 189])
v1 = np.array([53, 88, 79, 102, 198, 213])
w2 = np.array([112, 145, 165, 156, 198, 256])
v2 = np.array([145, 189, 199, 212, 215, 278])

import matplotlib.pyplot as plt
from modules.mid_market import single_sided_ida

z = np.linspace(1, 400, 400)

fig, ax = plt.subplots(1,1)
ax.plot(z, [single_sided_ida(a, v1, x) - single_sided_ida(b, w1, x) for x in z], label="lower")
ax.plot(z, [single_sided_ida(a, v2, x) - single_sided_ida(b, w2, x) for x in z], label="normal")
ax.legend()
plt.show()

### Total Margin

Basic margin:

In [ ]:
f"Ccy: {df['basic margin'].sum():,.0f}"

Volume add-on adjusted for liquidity:

In [ ]:
f"Ccy: {df['volume add on'].sum() * 2:,.0f}"

Correlation add-on adjusted for liquidity:

In [ ]:
f"Ccy: {(covar - covar_zerocorr) * 2:,.0f}"

Total Margin:

In [ ]:
tm = df["basic margin"].sum() + df["volume add on"].sum() * 2 + (covar - covar_zerocorr) * 2
f"Ccy: {tm:,.0f}"

# Alternative Example

Specify the risk and mid-market NPV of the requested trade.

In [ ]:
portfolio = Portfolio([
    Swap(datetime(2023,1,1), 12*4, 12, 12, notional=-100e6, fixed_rate=1.5),
    Swap(datetime(2027,5,21), 12*9, 12, 12, notional=45e6, fixed_rate=2.0),
    Swap(datetime(2028,9,16), 12*14, 12, 12, notional=21e6, fixed_rate=1.9)
])
portfolio.risk(s_cv)

In [ ]:
f"Ccy: {portfolio.npv(s_cv).real:,.0f}"

Get the current market-maker risk inventory

In [ ]:
S_inv = np.array([[1000, 35000, -10000, -35000]]).transpose()
x_inv = portfolio.model_margin(c, A, S_inv)[1][:, np.newaxis]

In [ ]:
ret = portfolio.model_margin(c, A, portfolio.risk(s_cv))
df = pd.DataFrame({
    "2Y": [1, 0, 0, 0, -1, -1, -1, 0, 0, 0, -1, 0],
    "5Y": [0, 1, 0, 0, 1, 0, 0, -1, -1, 0, 2, -1],
    "10Y": [0, 0, 1, 0, 0, 1, 0, 1, 0, -1, -1, 2],
    "30Y": [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, -1],
    "margin": [0.4, 0.5, 0.5, 0.6, 0.25, 0.45, 0.85, 0.25, 0.7, 0.55, 0.5, 0.6],
}, index=["2Y", "5Y", "10Y", "30Y", "2s5s", "2s10s", "2s30s", "5s10s", "5s30s", "10s30s", "2s5s10s", "5s10s30s"])
df["x"], x = ret[1], ret[1][:, np.newaxis]
df["x_inv"] = x_inv[:, 0]
df["basic margin"] = df["margin"] * np.abs(df["x"])
df["volume add on"] = 2/3000 * (np.abs((x_inv+x))**1.5 - np.abs(x_inv)**1.5)
df.style.format("{:,.1f}")

In [ ]:
covar = np.matmul(np.matmul(x.T, Q_model), x)[0][0] ** 0.5
covar

In [ ]:
covar_zerocorr = np.matmul(np.matmul(x.T, np.diag(np.diagonal(Q_model))), x)[0][0] ** 0.5
covar_zerocorr

The total we charge for this swap package is

In [ ]:
auto_margin = df["basic margin"].sum() + df["volume add on"].sum() + (covar - covar_zerocorr) * 0.15
f"Ccy: {auto_margin:,.0f}"

The price show to the customer is (negative is customer pays cash)

In [ ]:
f"Ccy: {(portfolio.npv(s_cv) - auto_margin).real:,.0f}"

This does not include an off-the-run charge which is a major component of risk in this package.